In [1]:
#dev/python3
import numpy as np
import itertools

In [2]:
file = open('T10I4D100K.dat', 'r')

data = []
for line in file:
    x = list(map(int, line.split(" ")[:-1]))
    data.append(set(x))

numItems = len(data)


In [3]:
#count number of occurences per item (C1 + count)
def generateC1(dataSet):
    productDict = {}
    for basket in dataSet:
        for product in basket:
            if product not in productDict:
                productDict[product] = 1
            else:
                productDict[product] = productDict[product] + 1
    return productDict

In [4]:
#eliminate items that have a support under the threshold 
def filterBySupport(d, support):
    return {k:v for k,v in d.items() if d[k]/numItems > support}

In [5]:
def generateCk(lk, k, dataset):
    
    countDict = {}
    
    if k > 2:
        lk = {item for t in lk for item in t}
        #print("lk len: ", len(lk))

    
    for basket in dataset:
        
        intersection = basket.intersection(lk)
        #sub = basket - intersection
        #newcanedidates = {(intersection + item) for item in sub}
        
        if len(intersection) > k:
            newcandidates = itertools.combinations(intersection, k)
            
            for c in newcandidates:

                if c not in countDict:
                    countDict[c] = 1
                else:
                    countDict[c] +=1
    
    return countDict
                

In [54]:
#test

C1 = generateC1(data)
L1 = filterBySupport(C1, 0.01)
#print(L1)
#print(set(L1.keys()))
C2 = generateCk(set(L1.keys()), 2, data)
L2 = filterBySupport(C2, 0.01)
L2

{(704, 39): 1104,
 (704, 825): 1099,
 (39, 825): 1137,
 (789, 829): 1101,
 (368, 829): 1138,
 (217, 346): 1279,
 (682, 368): 1155,
 (227, 390): 1040}

In [55]:
C3 = generateCk(set(L2.keys()), 3, data)
L3 = filterBySupport(C3, 0.01)
L3

lk len:  11


{}

In [11]:
def generateLK(dataset, support):
    
    dataset = dataset.copy()
    
    L = []
    C = []
    
    C.append(generateC1(dataset))
    L.append(filterBySupport(C[0], support))
    
    for k in range(1, 10):
        
        #print("C: ", k-1,  len(C[-1].keys()))
        #print("L: ", k-1, len(L[-1].keys()))
        
        C.append(generateCk(set(L[-1].keys()), k+1, dataset))
        L.append(filterBySupport(C[-1], support))
        
                  
        if len(L[len(L)-1]) == 0: 
            break
        
    return L,C
        

In [12]:
L, C = generateLK(data, 0.005)
print("candidates generated")

candidates generated


In [13]:
for k in range(len(L)):
    
    print( "round ", k+1, " : ",  list(L[k])[0:10], "\n")

round  1  :  [448, 834, 164, 775, 328, 687, 240, 368, 274, 561] 

round  2  :  [(448, 538), (704, 39), (704, 825), (39, 825), (708, 978), (708, 883), (978, 883), (883, 853), (782, 529), (674, 720)] 

round  3  :  [(704, 39, 825), (623, 853, 571), (623, 853, 795), (801, 392, 862), (579, 842, 411), (579, 842, 572), (579, 411, 572), (803, 842, 411), (803, 842, 572), (803, 411, 572)] 

round  4  :  [(803, 842, 411, 572), (579, 842, 411, 572), (290, 969, 458, 208), (290, 969, 458, 888), (290, 969, 208, 888), (290, 458, 208, 888), (969, 458, 208, 888), (33, 515, 217, 346), (33, 515, 217, 283), (33, 515, 346, 283)] 

round  5  :  [] 



In [14]:
for k in range(len(C)):
    
    print( "round ", k+1, " : ",  list(C[k])[0:10], "\n")

round  1  :  [448, 834, 164, 775, 328, 687, 240, 368, 274, 561] 

round  2  :  [(448, 834), (448, 164), (448, 775), (448, 328), (448, 687), (448, 240), (448, 368), (448, 274), (448, 561), (448, 52)] 

round  3  :  [(448, 834, 775), (448, 834, 240), (448, 834, 368), (448, 834, 274), (448, 834, 561), (448, 834, 52), (448, 834, 825), (448, 834, 538), (448, 775, 240), (448, 775, 368)] 

round  4  :  [(704, 39, 825, 857), (704, 39, 825, 895), (704, 39, 857, 895), (704, 825, 857, 895), (39, 825, 857, 895), (682, 782, 529, 947), (682, 782, 529, 185), (682, 782, 947, 185), (682, 529, 947, 185), (782, 529, 947, 185)] 

round  5  :  [(290, 969, 458, 208, 888), (290, 969, 458, 208, 217), (290, 969, 458, 888, 217), (290, 969, 208, 888, 217), (290, 458, 208, 888, 217), (969, 458, 208, 888, 217), (546, 947, 411, 661, 217), (546, 947, 411, 661, 923), (546, 947, 411, 217, 923), (546, 947, 661, 217, 923)] 

